In [207]:
import pandas
from googleapiclient.discovery import build
import warnings 
warnings.filterwarnings('ignore')
CAR_TYPE='i30'

In [208]:
api_key = ''
video_id = "EOCDIzUKRwc"
 
comments = list()
api_obj = build('youtube', 'v3', developerKey=api_key)
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute() 

while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
         
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
                
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break
    
df = pandas.DataFrame(comments)
df.shape

(222, 4)

In [209]:
df = pandas.DataFrame(comments)
df.columns=['text', 'ID', 'PublishedAt', 'LikeCnt']
texts = df.text.to_list()
sentiments = []
for text in texts:
    sentiments.append(TextBlob(text).sentiment.polarity)
df['sentiment'] = sentiments

In [210]:
file_path = f'../W2/{CAR_TYPE}_review_data.csv'
df.to_csv(file_path)

,text,ID,PublishedAt,LikeCnt,sentiment
0,Do you prefer an i30 n or a VW Golf GTI? Leave...,@WheelsAustralia,2021-09-14T06:25:31Z,9,0.000000
1,I’m a staunch VW fan. Couldn’t wait for the Mk...,@nickgentile4574,2021-09-14T09:01:53Z,9,0.360000
2,@@nickgentile4574 totally agree Nick. I own a ...,@alexinwood2723,2021-09-14T11:44:16Z,5,0.307143
3,"Neither, the Focus ST is still beat in this cl...",@351tgv,2021-09-17T12:13:38Z,3,0.000000
4,"On the eLSD, it&#39;s not an e diff. It&#39;s ...",@airii,2021-09-17T15:47:28Z,0,0.600000
...,...,...,...,...,...
217,@@Clarky_AU that’s exactly what I thought whil...,@WheelsAustralia,2021-09-14T05:50:57Z,3,0.246635
218,@@Clarky_AU &quot;Big difference between the 2...,@xXYannuschXx,2021-11-29T14:53:50Z,1,0.200000
219,Looks so good in red. Much better than the per...,@Aladdin5cave,2021-09-14T04:04:38Z,4,0.300000
220,Agreed! White is also mint,@alexinwood2723,2021-09-14T04:10:08Z,1,0.000000
